In [70]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path
import numpy as np
import pandas as pd
import pyvista as pv
from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
root_dir = Path("/home/max/Desktop/python_projects/SimJeb/")
metadata_dir = root_dir / "SimJEB_metadata"
meshes_dir = root_dir / "SimJEB_volmesh"

In [17]:
features_grid = pv.read(metadata_dir / "grid_rare.vtk")

In [27]:
for ds in ["train", "test"]:
    df = pd.read_csv(metadata_dir / f"{ds}_bracket_metadata.csv")
    category_df = df["category"]
    features = []
    for id_ in tqdm(df["id"]):
        mesh = pv.read(meshes_dir / f"{str(id_)}.vtk")
        cell_ids = features_grid.find_containing_cell(mesh.points)
        cell_ids = cell_ids[cell_ids >= 0]
        features_tmp = np.zeros((features_grid.n_cells,))
        features_tmp[cell_ids] = 1
        features_tmp_dict = {f"feature/{i}": features_tmp[i] for i in range(len(features_tmp))}
        features_tmp_dict["id"] = id_
        features.append(features_tmp_dict)
    features_df = pd.DataFrame(features)
    category_df = pd.DataFrame(category_df).join(features_df)
    category_df.to_csv(metadata_dir / f"{ds}_bracket_category.csv")

  0%|          | 0/83 [00:00<?, ?it/s]

In [30]:
train_df = pd.read_csv(metadata_dir / "train_bracket_category.csv")

In [31]:
train_df.head()

,Unnamed: 0,category,feature/0,feature/1,feature/2,feature/3,feature/4,feature/5,feature/6,feature/7,...,feature/190,feature/191,feature/192,feature/193,feature/194,feature/195,feature/196,feature/197,feature/198,id
0,0,flat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,241
1,1,block,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,281
2,2,block,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,170
3,3,flat,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,194
4,4,butterfly,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,497


In [62]:
le = preprocessing.LabelEncoder()
le.fit(train_df.category)
y_train = le.transform(train_df.category)

In [63]:
y_train.shape

(247,)

In [64]:
X_train = train_df.iloc[:,2:201].to_numpy()

In [65]:
X_train.shape

(247, 199)

In [66]:
clf = RandomForestClassifier()
estimator = clf.fit(X_train, y_train)

In [67]:
test_df = pd.read_csv(metadata_dir / "train_bracket_category.csv")

In [68]:
X_test = test_df.iloc[:,2:201].to_numpy()
y_test = le.transform(test_df.category)

In [69]:
y_test_pred = estimator.predict(X_test)

In [71]:
accuracy_score(y_test, y_test_pred)

0.9959514170040485